# Training a NER System with Transformers on CoNLL

Available models:

- BERT
- RoBERTa
- DistilBERT
- CamemBERT
- XLM-RoBERTa



In [1]:
import pandas as pd

from simpletransformers.ner.ner_model import NERModel

## Training a Toy Example

In [2]:
# toy dataset in BIO format
train_data = [
    [0, "My", "O"],
    [0, "friend", "O"],
    [0, "Pedro", "B-PER"],
    [0, "works", "O"],
    [0, "in ", "O"],
    [0, "the", "O"],
    [0, "European", "B-ORG"],
    [0, "Comission", "I-ORG"],
    [0, ".", "O"],
    [1, "Pedro", "B-PER"],
    [1, "was", "O"],
    [1, "born", "O"],
    [1, "in ", "O"],
    [1, "Madrid", "B-LOC"],
    [1, ".", "O"],
    [2, "Madrid", "B-LOC"],
    [2, "is", "O"],
    [2, "the", "O"],
    [2, "capital", "O"],
    [2, "of", "O"],
    [2, "Spain", "B-LOC"],
    [2, ".", "O"],
]

train_df = pd.DataFrame(train_data, columns=["sentence_id", "words", "labels"])

eval_data = [
    [0, "My", "O"],
    [0, "friend", "O"],
    [0, "Pedro", "B-PER"],
    [0, "works", "O"],
    [0, "in ", "O"],
    [0, "the", "O"],
    [0, "European", "B-ORG"],
    [0, "Comission", "I-ORG"],
    [0, ".", "O"],
    [1, "I", "O"],
    [1, "was", "O"],
    [1, "born", "O"],
    [1, "in ", "O"],
    [1, "Madrid", "B-LOC"],
    [1, ".", "O"],
]

eval_df = pd.DataFrame(eval_data, columns=["sentence_id", "words", "labels"])

In [6]:
# configuration
args = {
    "output_dir": "outputs/",
    "cache_dir": "cache_dir/",
    "fp16": False,
    "fp16_opt_level": "O1",
    "max_seq_length": 128,
    "train_batch_size": 32,
    "gradient_accumulation_steps": 1,
    "eval_batch_size": 8,
    "num_train_epochs": 10,
    "weight_decay": 0,
    "learning_rate": 4e-5,
    "adam_epsilon": 1e-8,
    "warmup_ratio": 0.06,
    "warmup_steps": 0,
    "max_grad_norm": 1.0,
    "logging_steps": 50,
    "save_steps": 2000,
    "overwrite_output_dir": True,
    "reprocess_input_data": False,
    "evaluate_during_training": False,
    #"process_count": cpu_count() - 2 if cpu_count() > 2 else 1,
    "n_gpu": 1,
    "wandb_project": "nlp-exercises",
}

In [7]:
# Create a NERModel
model = NERModel("bert", "bert-base-cased", use_cuda=False, args=args)

# Train the model
model.train_model(train_df)

# Evaluate the model
result, model_outputs, predictions = model.eval_model(eval_df)

Features loaded from cache at cache_dir/cached_train_bert_128_9_3


Running loss: 2.276351


Running loss: 2.280019


Running loss: 1.849380


Running loss: 1.455270


...

Running loss: 0.944153


Running loss: 0.826334


Running loss: 0.764017


Running loss: 0.695451

Training of bert model complete. Saved to outputs/.
Converting to features started.



{'eval_loss': 0.7346433401107788, 'precision': 0, 'recall': 0.0, 'f1_score': 0}


In [ ]:
# Predictions on arbitary text strings
predictions, raw_outputs = model.predict(["I live in Madrid."])

print(predictions)

## Training a NER Model on CoNLL

In [8]:
# configuration
args = {
    "output_dir": "outputs/",
    "cache_dir": "cache_dir/",
    "fp16": False,
    "fp16_opt_level": "O1",
    "max_seq_length": 128,
    "train_batch_size": 32,
    "gradient_accumulation_steps": 1,
    "eval_batch_size": 8,
    "num_train_epochs": 10,
    "weight_decay": 0,
    "learning_rate": 4e-5,
    "adam_epsilon": 1e-8,
    "warmup_ratio": 0.06,
    "warmup_steps": 0,
    "max_grad_norm": 1.0,
    "logging_steps": 50,
    "save_steps": 2000,
    "overwrite_output_dir": True,
    "reprocess_input_data": False,
    "evaluate_during_training": False,
    # "process_count": cpu_count() - 2 if cpu_count() > 2 else 1,
    "n_gpu": 1,
    "wandb_project": "nlp-exercises",
}

In [9]:
# Create a NERModel
model = NERModel("bert", "bert-base-cased", args=args)

# Train the model
model.train_model("../datasets/conll/train.txt")

Converting to features started.


Running loss: 0.829206

/opt/conda/envs/nlp/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:224: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 0.036342


Running loss: 0.043991


...

Running loss: 0.000504


Running loss: 0.002077


Running loss: 0.000254


Running loss: 0.001528


Running loss: 0.000080

Training of bert model complete. Saved to outputs/.


In [10]:
# Evaluate the model
result, model_outputs, predictions = model.eval_model("../datasets/conll/test.txt")

Converting to features started.



{'eval_loss': 0.1724665458686403, 'precision': 0.9068550497121926, 'recall': 0.9211552090715804, 'f1_score': 0.9139491957458029}


In [ ]:
# Check predictions
print(predictions[:5])

In [12]:
model.predict(["Peter works for the European Commission."])

Converting to features started.


([[{'Peter': 'B-PER'},
   {'works': 'O'},
   {'for': 'O'},
   {'the': 'O'},
   {'European': 'B-ORG'},
   {'Commission.': 'I-ORG'}]],
 array([[[ 9.30679   , -0.6898497 , -2.2677052 , ..., -0.90831643,
          -1.3977323 , -1.6333656 ],
         [-1.0378084 ,  0.283668  , -1.7377313 , ..., -2.3731523 ,
          -0.01672737, -1.3929404 ],
         [11.03432   , -1.7618532 , -1.1217908 , ..., -0.59245723,
          -2.409513  , -1.8573332 ],
         ...,
         [ 5.1532273 ,  0.10352939, -1.9668819 , ..., -1.6411906 ,
           0.05138788, -2.244919  ],
         [ 1.8262932 ,  0.5128787 , -1.5700084 , ..., -1.5704885 ,
           0.58235383, -1.7538601 ],
         [ 2.0302572 ,  0.36348754, -1.6550361 , ..., -1.7465127 ,
           0.5736985 , -1.7942278 ]]], dtype=float32))

## Load a Model after Training

In [ ]:
model = NERModel("bert", "outputs/", use_cuda=False, args={})

In [ ]:
model.predict(["Pedro lives in Madrid."])

## A Quick Demo

We can create magic cells in Notebooks using IPython magic commands.

In [ ]:
def load_model(
    model_architecture: str, directory: str = "outputs/", use_cuda: bool = False, **kwargs
):
    """Loads a pre-trained model"""
    model = NERModel(model_architecture, directory, use_cuda=use_cuda, args=kwargs)
    return model

In [ ]:
model = load_model("bert")

In [13]:
from IPython.core.magic import register_cell_magic

@register_cell_magic
def ner(line, text):
    """Prints predictons of a NER model"""
    predictions, raw_outputs = model.predict([text])
    return predictions[0]

In [15]:
%%ner
Elon Musk dreams of going to Mars.

Converting to features started.


[{'Elon': 'B-PER'},
 {'Musk': 'I-PER'},
 {'dreams': 'O'},
 {'of': 'O'},
 {'going': 'O'},
 {'to': 'O'},
 {'Mars.': 'B-LOC'}]